<a href="https://colab.research.google.com/github/dhivyapm/divya/blob/master/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I found the information about how the model works in this website
https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01

In [2]:
import pandas as pd # to read the dataset and convert the dataset to DataFrame which is in table format
import numpy as np # numpy is for computing, here I'm using this package for randomly select the rows from the dataset
import random # to randomly select the datapoints from dataset
df = pd.read_csv('/content/drive/My Drive/satelite.csv')
print(df.shape)

(6435, 37)


In [0]:
df_copy  = df.copy()

In [0]:
df1 = df_copy.drop('Column37',axis=1)

In [5]:
features = list(df1.columns)
random.seed(30) # If you want to generate the same number every time,before doing the shuffle
df = df.sample(frac=1)# shuffle the dataset
df.shape

(6435, 37)

In [6]:
#train_test_Split
train_size = int(0.9 * len(df)) #train_size =5791
X_train = df[features][:train_size] # X_train takes the values of the columns from 1 to 36 from the dataset upto the train_size rows(5791 rows)
y_train = df['Column37'][:train_size] #y_train takes the last column values which is label upto the train_size(5791 rows)
X_test = df[features][train_size:] # X_test takes the values of the columns from 1 to 36 starting from rows 5791 upto the dataset length which is 6435
y_test = df['Column37'][train_size:] # y_test takes the last column values which is label starting from rows 5791 upto the dataset length which is 6435
print(X_train)

      Column1  Column2  Column3  ...  Column34  Column35  Column36
2130       75       87       93  ...        91        97        75
4794       71       83       89  ...        84        82        68
305        93      115      114  ...        57        60        45
3293       46       32      117  ...        49       101       101
4922       67      108      119  ...       106       115        98
...       ...      ...      ...  ...       ...       ...       ...
4080       67       79       82  ...        79        79        59
4585       63       81       82  ...        81        82        61
3880       64       75       79  ...        75        80        59
232        79       87       89  ...        88        93        72
5721       55       87      104  ...        73        90        75

[5791 rows x 36 columns]


In [0]:
#calculating the bootstrap that is taking sample from train set and left out samples in dataset will be taken as out-of-bag which is used to test the model performance
def n_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True)) #it as the index of the sample selected from the training set
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices] # it as the index of the left over samples in the training set
    X_bootstrap = X_train.iloc[bootstrap_indices].values # here it takes the values of the specific index from the training
    y_bootstrap = y_train.iloc[bootstrap_indices].values # it takes the array elements of the specific indices(labels)
    X_oob = X_train.iloc[oob_indices].values # it takes the values of the left over samples in the training set
    y_oob = y_train.iloc[oob_indices].values # it takes the array elements from the left over samples in the training set(labels)
    return X_bootstrap, y_bootstrap, X_oob, y_oob # returns all the values of bootstrap and oob

In [8]:
X_bootstrap,y_bootstrap,X_oob,y_oob = n_bootstrap(X_train,y_train)
print(X_bootstrap,y_bootstrap)
print(X_oob,y_oob)

[[ 71  75  77 ...  77  86  60]
 [ 53  52  71 ...  54  74  62]
 [ 66 109 117 ...  88 105  83]
 ...
 [ 56  84  93 ...  66  82  72]
 [ 89 106 110 ... 103 104  81]
 [ 63  68  70 ...  73  75  57]] [7 5 1 ... 1 3 7]
[[ 97 116 113 ... 112 114  90]
 [ 48  32 123 ...  40 105 113]
 [ 52  73  90 ...  77  94  76]
 ...
 [ 68  77  82 ...  83  85  62]
 [ 63  81  82 ...  81  82  61]
 [ 79  87  89 ...  88  93  72]] [3 2 1 ... 7 7 7]


In [0]:
# calculate the Entropy(measure of disorder)
#To which group does this sample belongs to
#Entropy controls how a Decision Tree decides to split the data.
#where entropy takes in a probability of a class within a node 
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [0]:
#calculate the information gain to compare the entropy before and after split
#Information_gain takes in a list of the classes from the left and right child and returns the information gain of that particular split.
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent) #calculate the entropy of parent node
    IG_l = entropy(p_left) #calculate the entropy of left child node
    IG_r = entropy(p_right) #calculate the entropy of right child node
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r #return the information gain of parent and childs